# NT + ESM-2 ZERO-SHOT Ver


### 1) Split parquet into train/val/test (chr20/21 as test)


In [1]:
from split_data import main as split_main
from config import RAW_PARQUET
split_main(RAW_PARQUET)

Saved:
  train: 114568 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\train.parquet
  val  : 20219 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\val.parquet
  test : 4562 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\test.parquet


### 2) Precompute embeddings (NT + ESM-2)


In [2]:
%env TOKENIZERS_PARALLELISM=false
from precompute_embeddings import main as emb_main
emb_main()

env: TOKENIZERS_PARALLELISM=false


c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
Loading NT...


Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading ESM-2...


...

[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\train_embeddings.pt
Embedding split d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\val.parquet (20219 rows)


100%|██████████| 5055/5055 [02:56<00:00, 28.71it/s]


[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\val_embeddings.pt
Embedding split d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\test.parquet (4562 rows)


100%|██████████| 1141/1141 [00:39<00:00, 28.68it/s]


[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\test_embeddings.pt


### 3) Train fusion classifier

Để theo dõi tiến độ train trong tensorboard:

- Mở terminal -> nhập: tensorboard --logdir train/task3_variant_prediction/runs -> truy cập vào link được in ra


In [ ]:
EXP_NAME = "experiment_8_cross_attn_diff"

In [ ]:
from train import train
from config import LR, EPOCHS, BATCH_SIZE, PATIENCE, DROPOUT, SEED, PROJ_DIM, FUSION_HIDDEN, WEIGHT_DECAY, MODE, FUSION_METHOD, FEATURE_MODE
import argparse

parser = argparse.ArgumentParser(description="Train fusion classifier with enhanced features.")
parser.add_argument("--mode", type=str, default= MODE, choices=['dna','prot',MODE])
parser.add_argument("--fusion_method", type=str, default= 'cross_attn', choices=[FUSION_METHOD, 'cross_attn'])
parser.add_argument("--feature_mode", type=str, default= 'diff', choices=[FEATURE_MODE, 'ref_alt', 'diff', 'ref', 'alt'])
parser.add_argument("--proj_dim", type=int, default=PROJ_DIM)
parser.add_argument("--fusion_hidden", type=list, default=FUSION_HIDDEN)
parser.add_argument("--lr", type=float, default=1e-4, help="Learning rate")
parser.add_argument("--epochs", type=int, default=EPOCHS)
parser.add_argument("--batch_size", type=int, default=BATCH_SIZE)
parser.add_argument("--patience", type=int, default=PATIENCE)
parser.add_argument("--dropout", type=float, default=0.3)
parser.add_argument("--weight_decay", type=float, default=WEIGHT_DECAY, help="L2 regularization lambda")
parser.add_argument("--seed", type=int, default=SEED)
parser.add_argument("--exp_name", type=str, default=EXP_NAME, help="Experiment name (auto-generated if not provided)")
parser.add_argument("--log_dir", type=str, default=None, help="TensorBoard log directory (auto: runs/exp_name)")

args = parser.parse_args([])

result = train(args)
result

TRAINING CONFIGURATION:
  Experiment Name: experiment_7
  Device: cuda
  Mode: both
  Learning Rate: 0.0001
  Epochs: 30
  Batch Size: 128
  Patience: 5
  Dropout: 0.3
  Seed: 42
  Proj Dim: 512
  Fusion Hidden: [512, 256]
  Log Dir: runs\experiment_7
  Experiment Dir: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_7

============================== MODEL SUMMARY ==============================
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
FusionClassifier                         [128, 1280]               [128]                     --                        --                        --
├─ModalityProjector: 1-1                 [128, 1280]               [128, 512]                --                        --                        --
│    └─Sequential: 2-1                   [128, 3840]               [128, 512]                --       

[1] Train Loss: 0.1809 | Val Loss: 0.1490 | Train Acc: 0.9303 | Val Acc: 0.9423
--> Saved best model checkpoint to d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\best_fusion_model.pt


[2] Train Loss: 0.1380 | Val Loss: 0.1505 | Train Acc: 0.9471 | Val Acc: 0.9432


[3] Train Loss: 0.1202 | Val Loss: 0.1506 | Train Acc: 0.9538 | Val Acc: 0.9439


[4] Train Loss: 0.1030 | Val Loss: 0.1678 | Train Acc: 0.9609 | Val Acc: 0.9394


[5] Train Loss: 0.0844 | Val Loss: 0.1801 | Train Acc: 0.9684 | Val Acc: 0.9421


[6] Train Loss: 0.0569 | Val Loss: 0.2143 | Train Acc: 0.9793 | Val Acc: 0.9426
Early stopping triggered.

--- Testing with Best Model ---


[TEST] Loss: 0.1559 | AUC: 0.9350 | MCC: 0.8720 | Acc: 0.9373 | Spec: 0.9518
[TEST] Balanced Acc: 0.9350 | F1_macro: 0.9359 | Precision: 0.9352 | Recall: 0.9181

✓ Experiment saved to: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_7
  - Config: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_7\config.json
  - Args: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_7\args.json
  - Results: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_7\results.json
  - Model: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_7\best_model.pt


{'acc': 0.9373081922531128,
 'auc': 0.9349783658981323,
 'balanced_acc': 0.9349783658981323,
 'f1_macro': 0.9359430074691772,
 'mcc': 0.872007429599762,
 'precision': 0.9352331757545471,
 'recall': 0.918107807636261,
 'specificity': 0.9518489837646484,
 'loss': 0.1559282810309251}

### 4) Xem lại kết quả experiments

Các experiments được lưu tự động trong `embeddings/experiments/<exp_name>/`


In [ ]:
# Xem danh sách tất cả experiments
import os
import json
import pandas as pd

exp_base_dir = "embeddings/experiments"
if os.path.exists(exp_base_dir):
    experiments = []
    for exp_name in os.listdir(exp_base_dir):
        exp_dir = os.path.join(exp_base_dir, exp_name)
        results_file = os.path.join(exp_dir, "results.json")
        if os.path.exists(results_file):
            with open(results_file, "r") as f:
                data = json.load(f)
                experiments.append({
                    "exp_name": exp_name,
                    "timestamp": data.get("timestamp", ""),
                    "test_auc": data["test_results"].get("auc", 0),
                    "test_acc": data["test_results"].get("acc", 0),
                    "test_mcc": data["test_results"].get("mcc", 0),
                    "test_balanced_acc": data["test_results"].get("balanced_acc", 0),
                    "test_f1_macro": data["test_results"].get("f1_macro", 0),
                    "test_precision": data["test_results"].get("precision", 0),
                    "test_specificity": data["test_results"].get("specificity", 0),
                    "test_recall": data["test_results"].get("recall", 0),
                    "best_val_loss": data.get("best_val_loss", 0),
                    "epochs": data.get("epochs_trained", 0),
                })
    
    if experiments:
        df = pd.DataFrame(experiments)
        df = df.sort_values("timestamp", ascending=False)
        print("Danh sách experiments:")
        print(df.to_string(index=False))
    else:
        print("Chưa có experiments nào.")
else:
    print(f"Thư mục {exp_base_dir} chưa tồn tại.")

Danh sách experiments:
    exp_name                  timestamp  test_auc  test_acc  test_mcc  test_balanced_acc  test_f1_macro  test_precision  test_recall  best_val_loss  epochs
experiment_7 2026-01-15T18:17:44.304576  0.934978  0.937308  0.872007           0.934978       0.935943        0.935233     0.918108       0.148991       6
experiment_6 2026-01-15T18:05:26.511124  0.929813  0.934239  0.865984           0.929813       0.932502        0.946888     0.897762       0.149198       7
experiment_5 2026-01-15T17:56:35.657209  0.782050  0.783209  0.561216           0.782049       0.780232        0.736562     0.773652       0.467511       8
experiment_4 2025-12-26T22:15:42.118193  0.932280  0.935993  0.869411           0.932280       0.934404        0.943796     0.905392       0.148757       7
experiment_3 2025-12-26T22:13:35.635218  0.934255  0.935993  0.869390           0.934255       0.934683        0.929231     0.921668       0.149948       7
experiment_2 2025-12-26T22:01:18.202990  

In [ ]:
# Xem chi tiết một experiment cụ thể
exp_name = EXP_NAME  # Thay bằng exp_name muốn xem

exp_dir = f"embeddings/experiments/{exp_name}"
if os.path.exists(exp_dir):
    print(f"=== Experiment: {exp_name} ===\n")
    
    # Xem config
    config_file = os.path.join(exp_dir, "config.json")
    if os.path.exists(config_file):
        with open(config_file, "r") as f:
            config = json.load(f)
        print("Config:")
        for k, v in config.items():
            print(f"  {k}: {v}")
        print()
    
    # Xem cấu trúc Model
    summary_file = os.path.join(exp_dir, "model_summary.txt")
    if os.path.exists(summary_file):
        print("Model Architecture Summary:")
        with open(summary_file, "r", encoding="utf-8") as f:
            print(f.read())
        print()

    # Xem results
    results_file = os.path.join(exp_dir, "results.json")
    if os.path.exists(results_file):
        with open(results_file, "r") as f:
            results = json.load(f)
        print("Results:")
        print(f"  Best Val Loss: {results.get('best_val_loss', 0):.4f}")
        print(f"  Test AUC: {results['test_results'].get('auc', 0):.4f}")
        print(f"  Test Acc: {results['test_results'].get('acc', 0):.4f}")
        print(f"  Test MCC: {results['test_results'].get('mcc', 0):.4f}")
        print(f"  Test Balanced Acc: {results['test_results'].get('balanced_acc', 0):.4f}")
        print(f"  Test F1_macro: {results['test_results'].get('f1_macro', 0):.4f}")
        print(f"  Test Precision: {results['test_results'].get('precision', 0):.4f}")
        print(f"  Test Specificity: {results['test_results'].get('specificity', 0):.4f}")
        print(f"  Test Recall: {results['test_results'].get('recall', 0):.4f}")
        print(f"  Epochs Trained: {results.get('epochs_trained', 0)}")
else:
    print(f"Experiment '{exp_name}' không tồn tại.")

=== Experiment: experiment_7 ===

Config:
  exp_name: experiment_7
  timestamp: 2026-01-15T18:16:50.000655
  mode: both
  lr: 0.0001
  epochs: 30
  batch_size: 128
  patience: 5
  dropout: 0.3
  seed: 42
  proj_dim: 512
  fusion_hidden: [512, 256]
  log_dir: runs\experiment_7

Model Architecture Summary:
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
FusionClassifier                         [128, 1280]               [128]                     --                        --                        --
├─ModalityProjector: 1-1                 [128, 1280]               [128, 512]                --                        --                        --
│    └─Sequential: 2-1                   [128, 3840]               [128, 512]                --                        --                        --
│    │    └─Linear: 3-1                  [128, 3840]               [128, 512]                1,

# NT + ESM-2 FINE-TUNED Ver
